In [ ]:
import sys, serial, argparse, time
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import os
#import matplotlib.animation as animation
import scipy
from scipy import signal
from collections import deque

In [ ]:
ser = serial.Serial(
    port='com3',
    baudrate=9600,
)

In [ ]:
del ser

# Real-time signal

### Initialization - Zero setting

#### sampling rate 변함 = 값이 커지거나 급변할 때 (터치, proximity sensing) 줄어듦 (9600 아닌 값들)
#### 9600 에서는 38, 39 유지함 => 38.5 Hz

In [ ]:
sensor1, sensor2, sensor3, sensor4, index  = [], [], [], [], []

init_time = 5  #sec
window_size, Sampling_freq = 19, 38.5

init_period = Sampling_freq * init_time


def init(given_init_period):
    init_s1, init_s2, init_s3, init_s4 = [], [], [], []
    print("First Initialization")
    
    for i in range(int(init_period)):
        
        tmp = ser.readline()
        s1, s2, s3, s4, idx = tmp.decode()[:5], tmp.decode()[6:11], tmp.decode()[12:17], tmp.decode()[18:23], tmp.decode()[23:]
        
        if i == int(init_period) -1:
            end_period = float(idx) - 5000000
        
        try:
            s1, s2, s3, s4, idx = (int(s1)-10000), (int(s2)-10000), (int(s3)-10000), (int(s4)-10000), (float(idx)-5000000)
            
            if True:
                sensor1.append(s1)
                sensor2.append(s2)
                sensor3.append(s3)
                sensor4.append(s4)
                
                init_s1.append(s1)
                init_s2.append(s2)
                init_s3.append(s3)
                init_s4.append(s4)

                index.append(idx)
                
        except:
            pass

    return np.mean(init_s1), np.mean(init_s2), np.mean(init_s3), np.mean(init_s4), end_period


offset_s1, offset_s2, offset_s3, offset_s4, init_end_time = init(init_period)

In [ ]:
for i in range(1,5):
    print(i)

In [ ]:
sensor1

### 이제 할것

1. 필터링 할 방법 선정 => 이것도 절대값 씌우기 => good 
- Raw 데이터에 MA 아니면
- Raw 데이터에 LPF => 연산도 높을 것 같고 항상 저주파 대역이 높다고 하기가 어려움 
= MA 가 더 직관적이고 나을 것 같음, LPF가 Amplitude는 좀 더 높음

2. Th 설정: 움직이지 않았을 때 그것보다 조금 커야 하고 detection 값 보다 작아야 함
- 데이터 추출: LR (B->D), RL (D->B), Up (A->C), Down (C->A), CW (A ~ D), CCW (A ~ D), 위에 위치 in each sensor (A, B, C, D)
- 우선 30번 씩

3. zero setting 이랑 Th 같이 5초 마다 업데이트
4. Th 가 detect 되었을 때는 업데이트 하면 안됨

In [ ]:
init_s1, init_s2, init_s3, init_s4 = [], [], [], []
init_lst = []

while True:
    if ser.readable():
        res = ser.readline()
        s1, s2, s3, s4, idx = res.decode()[:5], res.decode()[6:11], res.decode()[12:17], res.decode()[18:23], res.decode()[23:]
                        
        try:
            s1, s2, s3, s4, idx = (int(s1)-10000), (int(s2)-10000), (int(s3)-10000), (int(s4)-10000), (float(idx)-5000000)

        except:
            print(idx, " index has Error")
            pass
        
        if type(s1) == int and type(s2) == int and type(s3) == int and type(s4) == int and type(idx) == float: 
            index.append(idx)
            
            for i in range (1,5):
                globals()['init_s{}'.format(i)].append(globals()['s{}'.format(i)])
                globals()['sensor{}'.format(i)].append(globals()['s{}'.format(i)] - globals()['offset_s{}'.format(i)])

            if idx % init_period == 0:  #매번 5초 마다 init 실행
                init_lst.append(idx)
                for k in range (1,5):
                    globals()['offset_s{}'.format(k)] = np.mean(globals()['init_s{}'.format(k)])
                    
                print(idx, offset_s1, offset_s2, offset_s3, offset_s4)
                init_s1, init_s2, init_s3, init_s4 = [], [], [], []

In [ ]:
len(sensor1), len(sensor2), len(sensor3), len(sensor4), len(index), init_lst

In [ ]:
plt.figure(figsize=(14,7))
plt.plot(sensor1[1:], label='Sensor 1')
plt.plot(sensor2[1:], label='Sensor 2')
plt.plot(sensor3[1:], label='Sensor 3')
plt.plot(sensor4[1:], label='Sensor 4')
plt.legend(fontsize=13)

plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(14,7))

window_size = 19

series_s1, series_s2, series_s3, series_s4 = pd.Series(sensor1), pd.Series(sensor2), pd.Series(sensor3), pd.Series(sensor4)

s1_ma, s2_ma, s3_ma, s4_ma = series_s1.rolling(window=window_size).mean(), series_s2.rolling(window=window_size).mean(), series_s3.rolling(window=window_size).mean(), series_s4.rolling(window=window_size).mean()

plt.plot(s1_ma, label='Sensor 1')
plt.plot(s2_ma, label='Sensor 2')
plt.plot(s3_ma, label='Sensor 3')
plt.plot(s4_ma, label='Sensor 4')

#plt.ylim(-5,5)

plt.legend(fontsize=13)

plt.grid(True)
plt.show()

In [ ]:
j=0
plt.figure(figsize=(20,20))

for i in range(1,5):
    plt.subplot(4, 2, i)
    x = np.arange(len(globals()['sensor{}'.format(i)])/2)
    fft = np.fft.fft(globals()['sensor{}'.format(i)]) / len(globals()['sensor{}'.format(i)])
    fft_magnitude = np.abs(fft)
    fft_shift = np.fft.fftshift(fft_magnitude)    
    
    plt.title('FFT sensor %d'%i, fontsize=13)
    plt.stem(x[1:], fft_shift[int(len(fft_shift)/2)+1:])
    plt.grid(True)
    
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(18,5))
start_length = int(len(fft_shift)/2)+1
plt.plot(fft_shift[start_length:])
plt.xlabel('Frequency', fontsize=15)
plt.grid(True)

# LPF

In [ ]:
plt.figure(figsize=(20,20))
j,k=1,1

for i in range(1,9):
    plt.subplot(8, 2, i)
    
    if i%2 == 1:
        plt.plot(globals()['sensor{}'.format(j)])
        plt.title('Sensor %d'%j, fontsize=15)
        j = j+1
        
    if i%2 == 0:
        b = signal.firwin(window_size, cutoff=5, fs=Sampling_freq, pass_zero='lowpass')
        x1 = signal.lfilter(b, [1.0], globals()['sensor{}'.format(k)])

        plt.plot(x1)
        plt.title('LPF 20Hz sensor %d'%k, fontsize=15)
        k = k+1
        
    plt.xlabel('time (ms)')
    plt.ylabel('Volt (V)')
    plt.grid(True)
    
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(20,20))
j,k=1,1

for i in range(1,9):
    plt.subplot(8, 2, i)
    
    if i%2 == 1:
        b = signal.firwin(window_size, cutoff=5, fs=Sampling_freq, pass_zero='lowpass')
        x1 = signal.lfilter(b, [1.0], globals()['sensor{}'.format(k)])

        plt.plot(x1)
        plt.title('LPF 20Hz sensor %d'%k, fontsize=15)
        j = j+1
        
    if i%2 == 0:
        globals()['series_sensor{}'.format(k)] = pd.Series(globals()['sensor{}'.format(k)])
        globals()['MA_sensor{}'.format(k)] = globals()['series_sensor{}'.format(k)].rolling(window=19).mean()

        plt.plot(globals()['MA_sensor{}'.format(k)])
        plt.title('Sensor %d'%j, fontsize=15)
        k = k+1
        
    plt.xlabel('time (ms)')
    plt.ylabel('Volt (V)')
    plt.grid(True)
    
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(20,8))
for k in range(1,5):
    b = signal.firwin(window_size, cutoff=5, fs=Sampling_freq, pass_zero='lowpass')
    x1 = signal.lfilter(b, [1.0], globals()['sensor{}'.format(k)])
    plt.plot(x1**2)
    plt.grid(True)

In [ ]:
plt.figure(figsize=(20,8))
for k in range(1,5):
    b = signal.firwin(window_size, cutoff=5, fs=Sampling_freq, pass_zero='lowpass')
    x1 = signal.lfilter(b, [1.0], globals()['sensor{}'.format(k)])
    plt.plot(x1**2)
    plt.ylim(0,300)
    plt.xlim(200,1000)
    plt.grid(True)

In [ ]:
plt.figure(figsize=(20,8))
for k in range(1,5):
    b = signal.firwin(window_size, cutoff=5, fs=Sampling_freq, pass_zero='lowpass')
    x1 = signal.lfilter(b, [1.0], globals()['sensor{}'.format(k)])
    plt.plot(x1**2)
    plt.ylim(0,300)
    plt.xlim(1000,2000)
    plt.grid(True)

In [ ]:
plt.figure(figsize=(20,8))
for k in range(1,5):
    b = signal.firwin(window_size, cutoff=5, fs=Sampling_freq, pass_zero='lowpass')
    x1 = signal.lfilter(b, [1.0], globals()['sensor{}'.format(k)])
    plt.plot(x1**2)
    plt.ylim(0,300)
    plt.xlim(2000,3000)
    plt.grid(True)

In [ ]:
plt.figure(figsize=(20,8))
for k in range(1,5):
    b = signal.firwin(window_size, cutoff=5, fs=Sampling_freq, pass_zero='lowpass')
    x1 = signal.lfilter(b, [1.0], globals()['sensor{}'.format(k)])
    plt.plot(x1**2)
    plt.ylim(0,300)
    plt.xlim(3000,4000)
    plt.grid(True)

In [ ]:
plt.figure(figsize=(20,8))
window_size = 19

series_s1, series_s2, series_s3, series_s4 = pd.Series(sensor1), pd.Series(sensor2), pd.Series(sensor3), pd.Series(sensor4)
s1_ma, s2_ma, s3_ma, s4_ma = series_s1.rolling(window=window_size).mean(), series_s2.rolling(window=window_size).mean(), series_s3.rolling(window=window_size).mean(), series_s4.rolling(window=window_size).mean()

plt.plot(s1_ma**2, label='Sensor 1')
plt.plot(s2_ma**2, label='Sensor 2')
plt.plot(s3_ma**2, label='Sensor 3')
plt.plot(s4_ma**2, label='Sensor 4')

#plt.ylim(-5,5)

plt.legend(fontsize=13)

plt.grid(True)
plt.show()

In [ ]:

for i in range(len(s1_ma)):
    
    

In [ ]:
Final_raw_MA = []
Final_LPF = []

# 차분 신호 1: |t - (t-1)|

In [ ]:
for j in range(1,5):
    globals()['t_subtracted{}'.format(j)] = []
    x1 = globals()['sensor{}'.format(j)]
    for i in range(len(globals()['sensor{}'.format(j)])-1):
        globals()['t_subtracted{}'.format(j)].append(np.abs(x1[i] - x1[i+1]))
        
plt.figure(figsize=(20,15))
j=0

for i in range(1,5):
    j = j+1
    plt.subplot(4, 2, j)
    plt.plot(globals()['t_subtracted{}'.format(i)])
    plt.title('sensor %d: (t+1) - t'%i, fontsize=15)
    plt.xlabel('time (ms)')
    plt.ylabel('Volt (mV)')
    plt.grid(True)
    
plt.tight_layout()
plt.show()

# 차분 신호 2

### Horizontal A-C, Vertical  B-D

#### 좋은 것 같지 않음

In [ ]:
j=0
plt.figure(figsize=(15,6))

tmp = np.array(sensor1) - np.array(sensor3)
plt.subplot(2, 2, 1)
plt.plot(tmp)
plt.title('Sensor A - Sensor C', fontsize=15)
plt.grid(True)
plt.ylabel('Volt (mV)')
plt.xlabel('time (ms)')


tmp = np.array(sensor2) - np.array(sensor4)
plt.subplot(2, 2, 2)
plt.plot(tmp)
plt.title('Sensor B - Sensor D', fontsize=15)
plt.grid(True)
plt.ylabel('Volt (mV)')
plt.xlabel('time (ms)')

plt.tight_layout()
plt.show()

In [ ]:
손동작 했을 시 고유 신호  
1. FFt를 통해서 가장 잘나오는 대역 추출
2. 차분 신호
3. 차분 신호 2


=> 가장 잘되는 거 선정 후 
=>Zero setting
- Threshold 계산
- Dynamic Threshold => 정확도 평가
- Frame 추출
- 추출된 프레임 DL에 넣기

